In [1]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tqdm.notebook import tqdm
import numpy as np

In [2]:
from dataset import MultiModalDataset
from model import FoodItemTagModel

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models, transforms

In [4]:
base_path = "/Users/santhosh.mohan/Downloads/DSCVAssessment/assignments/food_item_tag"

In [5]:
image_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(300),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [6]:
def train_epoch(model, dataloader, loss_func, optim, device = 'cpu'):
    model.train()
    epoch_loss = 0
    total_size = 0
    #dataiter = iter(dataloader)
    #data = dataiter.next()
    #print(type(data))
    #print(len(data))
    for batch in tqdm(dataloader):
        image = batch['image'].to(device)
        price = batch['price'].to(device)
        label = batch['label'].to(device)
        text = batch['text']
        text['input_ids'] = text['input_ids'].to(device)
        text['attention_mask'] = text['attention_mask'].to(device)
        optim.zero_grad()
        with torch.set_grad_enabled(True):
            output = model(image,text,price)
            loss = loss_func(output, label)
            loss.backward()
            optim.step()
        epoch_loss += loss.item() * data.size(0)
        total_size += data.size(0)
    epoch_loss = epoch_loss / total_size
    print(f"Training Loss - {epoch_loss}")

#     for image, text, price, label in tqdm(dataloader):
#         image = image.to(device)
#         new_txts = []
#         print(text)
#         print(text[0])
#         print(text[0][0])
#         for txt in text:
#             #print(txt)
#             new_txt = {x[0]:x[1] for x in txt}
#             #print(new_txt)
#             new_txt['input_ids'] = new_txt['input_ids'].to(device)
#             new_txt['attention_mask'] = new_txt['attention_mask'].to(device)
#             new_txts.append(new_txt)
#         price = price.to(device)
#         label = label.to(device)
#         optim.zero_grad()
#         with torch.set_grad_enabled(True):
#             output = model(image,new_txts,price)
#             loss = loss_func(output, label)
#             loss.backward()
#             optim.step()
#         epoch_loss += loss.item() * data.size(0)
#         total_size += data.size(0)
#     epoch_loss = epoch_loss / total_size
#     print(f"Training Loss - {epoch_loss}")

def validate_epoch(model, dataloader, loss_func, device = 'cpu'):
    model.eval()
    epoch_loss = 0
    total_size = 0
    #dataiter = iter(dataloader)
    #data = dataiter.next()
    for image, text, price, label in tqdm(dataloader):
#         print("loading data")
        image = image.to(device)
        text['input_ids'] = text['input_ids'].to(device)
        text['attention_mask'] = text['attention_mask'].to(device)
        price = price.to(device)
        label = label.to(device)
        with torch.set_grad_enabled(False):
            output = model(image,text,price)
            loss = loss_func(output, label)
        epoch_loss += loss.item() * data.size(0)
        total_size += data.size(0)
    epoch_loss = epoch_loss / total_size
    print(f"Validation Loss - {epoch_loss}")

In [7]:
def train_image_model(device='cpu', epochs = 10):
    model = FoodItemTagModel(512,512,48)
    model = model.to(device)
    loss_func = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[40,70, 90], gamma=0.15)
    training_dataset = MultiModalDataset(f"{base_path}/data/training_data.csv", f"{base_path}/imgs", image_transforms["train"])
    validation_dataset = MultiModalDataset(f"{base_path}/data/validation_data.csv", f"{base_path}/imgs", image_transforms["val"])


    training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=4,
                                                 shuffle=True, num_workers=0)
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=4,
                                                 shuffle=True, num_workers=0)#,collate_fn=lambda x: x)
    for epoch in range(epochs):
        print(f"Epoch - {epoch+1}")
        train_epoch(model, training_dataloader, loss_func, optimizer, device)
        validate_epoch(model, validation_dataloader, loss_func, device)
        scheduler.step()
    return model

In [8]:
model = train_image_model(epochs = 2)
torch.save(model.state_dict(), "model.pth")

Some weights of the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_le

Epoch - 1


AttributeError: 

### MultiModal Model

Since it is a baseline model, i am just using basing hashing for text features, also training image model separately

In [11]:
def train_multimodal_epoch(image_model, model, dataloader, loss_func, optim, device = 'cpu'):
    image_model.eval()
    model.train()
    epoch_loss = 0
    total_size = 0
    for image, text, label in tqdm(dataloader):
        image = image.to(device)
        text = text.to(device)
        label = label.to(device)
        optim.zero_grad()
        with torch.set_grad_enabled(False):
            image_features = image_model(image)
        
        with torch.set_grad_enabled(True):
            output = model(image_features, text)
            loss = loss_func(output, label)
            loss.backward()
            optim.step()
        epoch_loss += loss.item() * label.size(0)
        total_size += label.size(0)
    epoch_loss = epoch_loss / total_size
    print(f"Training Loss - {epoch_loss}")

def validate_multimodal_epoch(image_model, model, dataloader, loss_func, device = 'cpu'):
    image_model.eval()
    model.eval()
    epoch_loss = 0
    total_size = 0
    for image, text, label in tqdm(dataloader):
        image = image.to(device)
        text = text.to(device)
        label = label.to(device)     
        with torch.set_grad_enabled(False):
            image_features = image_model(image)
            output = model(image_features, text)
            loss = loss_func(output, label)
        epoch_loss += loss.item() * label.size(0)
        total_size += label.size(0)
    epoch_loss = epoch_loss / total_size
    print(f"Validation Loss - {epoch_loss}")

In [14]:
def train_multimodal_model(image_model, device='cpu',epochs = 10, resume_model = None):
    image_model = image_model.to(device)
    model = MultiModalModel(48, 61, 48)
    if resume_model:
        model.load_state_dict(resume_model)
    model = model.to(device)
    loss_func = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[40,70, 90], gamma=0.15)
    hashing = HashingVectorizer(n_features=20)
    df = pd.read_csv(f"{base_path}/data/training_data.csv")
    hashing.fit(df['name'])
    
    training_dataset = MultiModalDataset(f"{base_path}/data/training_data.csv", f"{base_path}/imgs", image_transforms["train"], hashing)
    validation_dataset = MultiModalDataset(f"{base_path}/data/validation_data.csv", f"{base_path}/imgs", image_transforms["val"], hashing)


    training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=512,
                                                 shuffle=True, num_workers=0)
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=512,
                                                 shuffle=True, num_workers=0)
    for epoch in range(epochs):
        print(f"Epoch - {epoch+1}")
        train_multimodal_epoch(image_model, model, training_dataloader, loss_func, optimizer, device)
        validate_multimodal_epoch(image_model, model, validation_dataloader, loss_func, device)
        scheduler.step()
    return model

In [13]:
img_model = models.alexnet(pretrained=True)
img_model.classifier[6] = nn.Linear(4096,48)
img_model.load_state_dict(torch.load("alexnet_model.pth"))
mul_model = train_multimodal_model(img_model, device='cpu',epochs = 10)


  0%|          | 0/16 [00:00<?, ?it/s]

Epoch - 1


  0%|          | 0/4 [00:00<?, ?it/s]

Training Loss - 0.6950360305638437


  0%|          | 0/16 [00:00<?, ?it/s]

...

Validation Loss - 0.6788387785108787
Epoch - 10


  0%|          | 0/4 [00:00<?, ?it/s]

Training Loss - 0.6783427005847646


100%|██████████| 4/4 [01:19<00:00, 19.83s/it]

Validation Loss - 0.6769292623200996


In [15]:
torch.save(mul_model.state_dict(), "multimodal_model.pth")

In [19]:
model_state = torch.load("multimodal_model.pth")
mul_model = train_multimodal_model(img_model, device='cpu',epochs = 20,resume_model = model_state)


  0%|          | 0/16 [00:00<?, ?it/s]

Epoch - 1


  0%|          | 0/4 [00:00<?, ?it/s]

Training Loss - 0.5145476912940407


  0%|          | 0/16 [00:00<?, ?it/s]

...

Validation Loss - 0.2189101742143318
Epoch - 20


  0%|          | 0/4 [00:00<?, ?it/s]

Training Loss - 0.21959111066183978


100%|██████████| 4/4 [01:12<00:00, 18.07s/it]

Validation Loss - 0.21862764035834864


In [20]:
torch.save(mul_model.state_dict(), "multimodal_model.pth")

In [23]:
hashing = HashingVectorizer(n_features=20)
df = pd.read_csv(f"{base_path}/data/training_data.csv")
hashing.fit(df['name'])

validation_dataset = MultiModalDataset(f"{base_path}/data/validation_data.csv", f"{base_path}/imgs", image_transforms["val"], hashing)

validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=512,
                                             shuffle=True, num_workers=0)


def validate_multimodal_epoch(image_model, model, dataloader, device = 'cpu'):
    image_model.eval()
    model.eval()
    PRED = []
    for image, text, label in tqdm(dataloader):
        image = image.to(device)
        text = text.to(device)
        label = label.to(device)     
        with torch.set_grad_enabled(False):
            image_features = image_model(image)
            output = model(image_features, text)
            print(output.shape)
            print(output.type)
            print(output)
            break
validate_multimodal_epoch(img_model, mul_model, validation_dataloader)

  0%|          | 0/4 [00:18<?, ?it/s]

torch.Size([512, 48])
<built-in method type of Tensor object at 0x7fae5131bcb0>
tensor([[-3.7887, -5.0942, -4.1025,  ..., -5.4269, -3.6412, -4.0881],
        [-2.7067, -3.6115, -2.9300,  ..., -3.8476, -2.5861, -2.8831],
        [-3.4922, -4.6908, -3.7853,  ..., -5.0030, -3.3606, -3.7574],
        ...,
        [-2.5565, -3.4077, -2.7526,  ..., -3.6117, -2.4274, -2.7098],
        [-3.6490, -4.9045, -3.9514,  ..., -5.2258, -3.5110, -3.9338],
        [-3.6611, -4.9119, -3.9616,  ..., -5.2395, -3.5195, -3.9461]])
